# スタッキングの実装
ここでは2値分類用のスタッキングのクラスを実装する。  
scikit-learnにもスタッキング用のクラスが存在するが、学習時にKFold CVをしていない模様。なので過学習気味になる恐れ。    
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html

In [3]:
%load_ext autoreload
%autoreload 2
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, label_binarize, OrdinalEncoder
# import statsmodels.api as sma
from sklearn.model_selection import train_test_split ,LeaveOneOut, cross_val_score, KFold, RepeatedKFold,StratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, log_loss, confusion_matrix,ConfusionMatrixDisplay, \
accuracy_score, precision_score, recall_score,precision_recall_curve,f1_score,roc_curve,auc,get_scorer_names,roc_auc_score
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage,dendrogram,fcluster
from sklearn import tree
from xgboost import XGBClassifier
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

%matplotlib inline
import matplotlib.pyplot as plt


In [22]:
# polarsでタイタニックデータを読み込み
df = pl.from_pandas(sns.load_dataset('titanic'))

# 欠損値は落とさなくてよい
df = df.drop_nulls()

# 学習データ、目的変数を定義
X = df.drop(['survived', 'alive'])
y = df.get_column('survived')

# カテゴリ変数のカラム名をリスト化
category_cols = X.select(pl.col([pl.Utf8, pl.Categorical, pl.Boolean])).columns

# ラベルエンコーディング（LabelEncoderではなく、OrdinalEncoderを使う）
oe = OrdinalEncoder()
# pandasで返ってくるように指定。polarsは指定できない模様
oe.set_output(transform='pandas')
# カテゴリ変数をエンコーディング。polars.DFはそのまま入れられないのでpandasに変換する。
X = X.with_columns( pl.from_pandas(oe.fit_transform(X.select(category_cols).to_pandas())) )

# hold-out
X_train, X_test, y_train, y_test = train_test_split(X.to_pandas(), y.to_pandas(), test_size=0.3, random_state=0)

## 実装前の確認

In [5]:
# kfoldの定義
cv = KFold(n_splits=5, shuffle=True, random_state=0)
estimator = XGBClassifier(early_stopping_rounds=10, learning_rate=0.01, eval_metric='auc',random_state=0)

In [15]:
%%time
X_pd = X.to_pandas()
y_pd = y.to_pandas()
y_pred_proba_li = []

# Layer1
for train_index, val_index in cv.split(X_pd):
    X_train, X_val = X_pd.iloc[train_index], X_pd.iloc[val_index]
    y_train, y_val = y_pd.iloc[train_index], y_pd.iloc[val_index]
    
    # モデル学習
    eval_set = [(X_val, y_val)]
    estimator.fit(X_train, y_train, eval_set=eval_set, verbose=True)
    # 検証データ(学習に使っていないデータ)に対する予測値算出
    y_pred_proba = estimator.predict_proba(X_val)

    # 予測値を追加していく
    y_pred_proba_li.append(y_pred_proba)

result = np.concatenate(y_pred_proba_li)

[0]	validation_0-auc:0.85264
[1]	validation_0-auc:0.85382
[2]	validation_0-auc:0.90184
[3]	validation_0-auc:0.86733
[4]	validation_0-auc:0.86838
[5]	validation_0-auc:0.86838
[6]	validation_0-auc:0.86838
[7]	validation_0-auc:0.86838
[8]	validation_0-auc:0.86838
[9]	validation_0-auc:0.86798
[10]	validation_0-auc:0.86838
[11]	validation_0-auc:0.86825
[0]	validation_0-auc:0.82661
[1]	validation_0-auc:0.82990
[2]	validation_0-auc:0.82910
[3]	validation_0-auc:0.82896
[4]	validation_0-auc:0.82896
[5]	validation_0-auc:0.82923
[6]	validation_0-auc:0.82937
[7]	validation_0-auc:0.82977
[8]	validation_0-auc:0.82977
[9]	validation_0-auc:0.82795
[10]	validation_0-auc:0.82795
[11]	validation_0-auc:0.82822
[0]	validation_0-auc:0.87184
[1]	validation_0-auc:0.87270
[2]	validation_0-auc:0.87385
[3]	validation_0-auc:0.87457
[4]	validation_0-auc:0.87514
[5]	validation_0-auc:0.87514
[6]	validation_0-auc:0.87514
[7]	validation_0-auc:0.87514
[8]	validation_0-auc:0.87514
[9]	validation_0-auc:0.88182
[10]	valid

○ メモ  
逐次concatenateする場合繰り返しconcatenateのオーバーヘッドがあるので、  
リストでまとめてconcatするよりも時間がかかるらしい。（大規模データの場合）

In [7]:
%%time
X_pd = X.to_pandas()
y_pd = y.to_pandas()
# 予測値を入れる初期の空の配列を作成
result_2 = np.empty((0, 2))

# Layer1
for train_index, val_index in cv.split(X_pd ,y_pd):
    X_train, X_val = X_pd.iloc[train_index], X_pd.iloc[val_index]
    y_train, y_val = y_pd.iloc[train_index], y_pd.iloc[val_index]
    
    # モデル学習
    eval_set = [(X_val, y_val)]
    estimator.fit(X_train, y_train, eval_set=eval_set, verbose=True)
    # 検証データ(学習に使っていないデータ)に対する予測値算出
    y_pred_proba = estimator.predict_proba(X_val)

    # 予測値を追加していく
    result_2 = np.concatenate((result, y_pred_proba))
    

[0]	validation_0-auc:0.85264
[1]	validation_0-auc:0.85382
[2]	validation_0-auc:0.90184
[3]	validation_0-auc:0.86733
[4]	validation_0-auc:0.86838
[5]	validation_0-auc:0.86838
[6]	validation_0-auc:0.86838
[7]	validation_0-auc:0.86838
[8]	validation_0-auc:0.86838
[9]	validation_0-auc:0.86798
[10]	validation_0-auc:0.86838
[11]	validation_0-auc:0.86825
[0]	validation_0-auc:0.82661
[1]	validation_0-auc:0.82990
[2]	validation_0-auc:0.82910
[3]	validation_0-auc:0.82896
[4]	validation_0-auc:0.82896
[5]	validation_0-auc:0.82923
[6]	validation_0-auc:0.82937
[7]	validation_0-auc:0.82977
[8]	validation_0-auc:0.82977
[9]	validation_0-auc:0.82795
[10]	validation_0-auc:0.82795
[0]	validation_0-auc:0.87184
[1]	validation_0-auc:0.87270
[2]	validation_0-auc:0.87385
[3]	validation_0-auc:0.87457
[4]	validation_0-auc:0.87514
[5]	validation_0-auc:0.87514
[6]	validation_0-auc:0.87514
[7]	validation_0-auc:0.87514
[8]	validation_0-auc:0.87514
[9]	validation_0-auc:0.88182
[10]	validation_0-auc:0.88182
[11]	valid

In [64]:
a = {}
a['x'] = np.array([1, 2, 3])

In [62]:
a

{'x': array([1, 2, 3])}

## 実装

In [8]:
# 呼び出し側
cv = KFold(n_splits=5, shuffle=True, random_state=0)

estimators = [
    ('DT', tree.DecisionTreeClassifier(max_depth=2)),
    ('XGB', XGBClassifier(early_stopping_rounds=10, learning_rate=0.01, eval_metric='auc',random_state=0)),
    ('LGBM', lgb.LGBMClassifier(boosting_type='goss', max_depth=5, random_state=0))
]

In [19]:
class MyStackingClassifierCV:
    def __init__(self, estimators, final_estimator=None, cv=None):
        self.estimators = estimators
        self.final_estimator = final_estimator
        self.cv = cv

    def fit(self, X, y):

        # Layer1の学習
        self.y_pred_proba_dict = {}
            
        for estimator_name, estimator in self.estimators:
            # モデル名と予測値のリストを対応させる。
            self.y_pred_proba_dict[estimator_name] = []
            # テストデータに対するLayer1での予測値格納用

            # Layer2へ渡す特徴量生成のための学習
            for train_index, val_index in self.cv.split(X):
                # 学習用データと予測値算出用データに分ける
                X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                y_train, y_val = y.iloc[train_index], y.iloc[val_index]
                # XGBoostのモデル学習
                if estimator_name == 'XGB':
                    eval_set = [(X_val, y_val)]
                    estimator.fit(X_train, y_train, eval_set=eval_set, verbose=True)
                elif estimator_name == 'LGBM':
                    # LightGBMのモデル学習
                    eval_set = [(X_val, y_val)]
                    callbacks = [lgb.early_stopping(stopping_rounds=10), lgb.log_evaluation()]
                    estimator.fit(X_train, y_train, eval_set=eval_set, callbacks=callbacks)
                else:
                    # 決定木のモデル学習
                    estimator.fit(X_train, y_train)

                # 学習に使わなかったデータに対する予測値を算出し、リストに追加
                self.y_pred_proba_dict[estimator_name].append(estimator.predict(X_val))

            # 全foldでの予測値を結合してそのモデルの最終的な予測値を算出
            self.y_pred_proba_dict[estimator_name] = np.concatenate(self.y_pred_proba_dict[estimator_name])

            # テストデータに対する予測のための学習
            X_train2, X_val2, y_train2, y_val2 = train_test_split(X, y, test_size=0.3, random_state=0)
            # XGBoostのモデル学習
            if estimator_name == 'XGB':
                eval_set = [(X_val2, y_val2)]
                estimator.fit(X_train2, y_train2, eval_set=eval_set, verbose=True)
            elif estimator_name == 'LGBM':
                # LightGBMのモデル学習
                eval_set = [(X_val2, y_val2)]
                callbacks = [lgb.early_stopping(stopping_rounds=10), lgb.log_evaluation()]
                estimator.fit(X_train2, y_train2, eval_set=eval_set, callbacks=callbacks)
            else:
                # 決定木のモデル学習
                estimator.fit(X_train2, y_train2)
        
        # Layer1での予測値をまとめる（Layer2へ渡す用）。array[[モデル1の予測結果, モデル2の予測結果・・・]]の形式。
        self.result_layer1 = np.stack(list(self.y_pred_proba_dict.values()), axis=-1)

        # Layer2の学習
        self.final_estimators.fit(self.result_layer1, y)

        return self.y_pred_proba_dict
                


In [20]:
model = MyStackingClassifierCV(estimators=estimators, cv=cv)

In [26]:
result_3 = model.fit(X_train, y_train)

[0]	validation_0-auc:0.80303
[1]	validation_0-auc:0.82121
[2]	validation_0-auc:0.82121
[3]	validation_0-auc:0.82121
[4]	validation_0-auc:0.82121
[5]	validation_0-auc:0.82121
[6]	validation_0-auc:0.81212
[7]	validation_0-auc:0.81212
[8]	validation_0-auc:0.78182
[9]	validation_0-auc:0.75152
[10]	validation_0-auc:0.73939
[0]	validation_0-auc:0.81818
[1]	validation_0-auc:0.81818
[2]	validation_0-auc:0.81818
[3]	validation_0-auc:0.81818
[4]	validation_0-auc:0.82955
[5]	validation_0-auc:0.82955
[6]	validation_0-auc:0.82955
[7]	validation_0-auc:0.82955
[8]	validation_0-auc:0.82955
[9]	validation_0-auc:0.81250
[10]	validation_0-auc:0.81250
[11]	validation_0-auc:0.81250
[12]	validation_0-auc:0.81250
[13]	validation_0-auc:0.81250
[14]	validation_0-auc:0.81250
[0]	validation_0-auc:0.83333
[1]	validation_0-auc:0.85417
[2]	validation_0-auc:0.85417
[3]	validation_0-auc:0.85417
[4]	validation_0-auc:0.85417
[5]	validation_0-auc:0.85417
[6]	validation_0-auc:0.85417
[7]	validation_0-auc:0.85417
[8]	vali

In [35]:
X_train

,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alone
125,1,1.0,42.0,1,0,52.5542,2.0,0.0,1.0,1.0,3.0,2.0,0.0
40,1,1.0,44.0,2,0,90.0000,1.0,0.0,1.0,1.0,2.0,1.0,0.0
108,1,0.0,22.0,0,2,49.5000,0.0,0.0,2.0,0.0,1.0,0.0,0.0
22,1,0.0,19.0,0,2,26.2833,2.0,0.0,2.0,0.0,3.0,2.0,0.0
165,1,1.0,46.0,0,0,79.2000,0.0,0.0,1.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,1,1.0,47.0,0,0,34.0208,2.0,0.0,1.0,1.0,3.0,2.0,1.0
67,1,1.0,29.0,1,0,66.6000,2.0,0.0,1.0,1.0,2.0,2.0,0.0
117,1,0.0,39.0,1,1,110.8833,0.0,0.0,2.0,0.0,2.0,0.0,0.0
47,1,0.0,58.0,0,1,153.4625,2.0,0.0,2.0,0.0,2.0,2.0,0.0


In [34]:
X_train.values

array([[ 1.,  1., 42., ...,  3.,  2.,  0.],
       [ 1.,  1., 44., ...,  2.,  1.,  0.],
       [ 1.,  0., 22., ...,  1.,  0.,  0.],
       ...,
       [ 1.,  0., 39., ...,  2.,  0.,  0.],
       [ 1.,  0., 58., ...,  2.,  2.,  0.],
       [ 1.,  0., 39., ...,  4.,  0.,  0.]])

In [31]:
result_3

{'DT': array([1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0]),
 'XGB': array([1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
        1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0]),
 'LGBM': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,

In [37]:
result_3.values()

dict_values([array([1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0]), array([1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0]), array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 

In [40]:
list(result_3.values())

[array([1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0]),
 array([1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
        1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,

In [41]:
np.stack(list(result_3.values()), axis=-1)

array([[1, 1, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 1],
       [1, 1, 1],
       [0, 0, 1],
       [1, 1, 1],
       [0, 1, 1],
       [1, 1, 1],
       [0, 1, 1],
       [0, 0, 1],
       [0, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 1, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [0, 0, 1],
       [0, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 0, 1],
       [1, 1, 1],
       [0, 1, 1],
       [0, 0, 1],
       [0, 1, 1],
       [0, 0, 1],
       [0, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [0, 1, 1],
       [1, 1, 1],
       [0, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 1, 1],
       [0, 0, 1],
       [0, 1, 0],
       [1,